**Anime Recommendation System via Collaborative Filtering**

MSCI 623 Project Spring 2020

In [ ]:
#Importing the colab drive library to get the dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
import scipy as sp
from google.colab import drive
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
import operator
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Importing Data**

In [ ]:
#Importing the required  rating dataset as well as anime dataset
dataset=pd.read_csv("/content/drive/My Drive/Anime_Recommend_System/anime_updated.csv")
rating =pd.read_csv("/content/drive/My Drive/Anime_Recommend_System/rating.csv")
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


**Data Pre-Processing**

In [ ]:
# Rating dataset has ample rows which have -1 ratings , thus replacing all the -1 values with NaN
rating.rating.replace(-1,np.nan,inplace = True)
rating.head()

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


In [ ]:
#Since most popular types of anime are TV, Movie and OVA , filtering for those types of animes
genre = ['TV', 'Movie','OVA']
filtered_Dataset = dataset[dataset.type.isin(genre)]
filtered_Dataset.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
# Now we have to merge both the ratings and the anime dataset using merge function and we rename the column for easy name convention
merged_dataframe = rating.merge(filtered_Dataset, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
merged_dataframe.rename(columns = {'rating_user':'user_rating'}, inplace = True)

In [ ]:
#We are limiting the dataframe length to 25,000 users as Google colab was running out of memory if we try to process more users
merged_dataframe=merged_dataframe[['user_id', 'name', 'user_rating']]
merged_sub_dataframe= merged_dataframe[merged_dataframe.user_id <= 25000]
merged_sub_dataframe.head()

,user_id,name,user_rating
0,1,Naruto,NaN
1,3,Naruto,8.0
2,5,Naruto,6.0
3,6,Naruto,NaN
4,10,Naruto,NaN


In [ ]:
#Now we need to create pivot table of the above sub-merged dataframe we acquired in the last frame, so that we get row for every user id and columns as Anime names
pivottable = merged_sub_dataframe.pivot_table(index=['user_id'], columns=['name'], values='user_rating')

In [ ]:
#Checking the shape of the pivot table and viewing the top few rows of the pivot table, how it looks like
print(pivottable.shape)
pivottable.head()

(23632, 7046)


name,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,&quot;Eiji&quot;,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//Gift,.hack//Liminality,.hack//Quantum,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,.hack//The Movie: Sekai no Mukou ni,009 Re:Cyborg,009-1,07-Ghost,0:08,1+2=Paradise,100%,1000-nen Joou: Queen Millennia,1001 Nights,11-nin Iru!,11eyes,12-gatsu no Uta,12-sai.,12-sai.: Chicchana Mune no Tokimeki,12-sai.: Chicchana Mune no Tokimeki 2nd Season,15 Bishoujo Hyouryuuki,15 Sonyeon Uju Pyoryugi,21-ji no Onna: Newscaster Katsuragi Miki,2x1,3 Choume no Tama: Onegai! Momo-chan wo Sagashite!!,3 Choume no Tama: Uchi no Tama Shirimasenka?,3-tsu no Kumo,30-sai no Hoken Taiiku,30th Gundam Perfect Mission,3Ping Lovers!☆Ippu Nisai no Sekai e Youkoso♪ The Animation,3x3 Eyes,3x3 Eyes Seima Densetsu,663114,...,Zettai Junpaku♡Mahou Shoujo,Zettai Junshu☆Kyousei Kozukuri Kyokashou!!,Zettai Karen Children,Zettai Karen Children Gentei Kaikin!! OVA Chou Sakidori Special!!,Zettai Karen Children OVA: Aitazousei! Ubawareta Mirai?,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zettai Muteki Raijin-Oh,Zettai Muteki Raijin-Oh (1992),Zettai Shougeki: Platonic Heart,Zettai Shounen,Zipang,Zoids,Zoids Fuzors,Zoids Genesis,Zoids Shinseiki/Zero,Zoku Gosenzo San&#039;e,Zoku Koihime,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,"Zone of the Enders: Dolores, I",Zone of the Enders: Idolo,Zukkoke Knight: Don De La Mancha,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,_Summer,eX-Driver,eX-Driver the Movie,eX-Driver: Nina &amp; Rei Danger Zone,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,gdgd Fairies Movie: tte Iu Eiga wa Dou kana...?,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Now as different users would rate differently thus we need to normalise the scores present in the pivot table
pivot_norm = pivottable.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

In [ ]:
#Now as seen from the EDA, there are lots of unrated content in the database thus we have to remove it
# Drop all columns containing only zeros representing users who did not rate
pivot_norm.fillna(0, inplace=True)
pivot_norm = pivot_norm.T
pivot_norm = pivot_norm.loc[:,(pivot_norm!= 0).any(axis=0)]

In [ ]:
pivot_sparse = sp.sparse.csr_matrix(pivot_norm.values)  #there is lot of sparcity in the matrix

**Implementing Item Based Collaborative System**

In [ ]:
#We are working upon Collaborative Filtering and thus we would try making the recommendation on item based similarity
item_similarity = cosine_similarity(pivot_sparse)

In [ ]:
# Inserting the similarity matricies into dataframe objects So that we can perform operations and can further start the main task of recommendation
item_sim_df = pd.DataFrame(item_similarity, index = pivot_norm.index, columns = pivot_norm.index)

In [ ]:
#Function to display top 10 similar animes given an anime name
def top_animes(anime_name):
    count = 1
    print('Similar shows to {} include:\n'.format(anime_name))
    for item in item_sim_df.sort_values(by = anime_name, ascending = False).index[1:11]:
        print('No. {}: {} '.format(count, item))
        count +=1  

In [ ]:
top_animes('Hajime no Ippo')

Similar shows to Hajime no Ippo include:

No. 1: Hajime no Ippo: New Challenger 
No. 2: Hajime no Ippo: Rising 
No. 3: Hajime no Ippo: Mashiba vs. Kimura 
No. 4: Great Teacher Onizuka 
No. 5: Hunter x Hunter (2011) 
No. 6: Major S5 
No. 7: One Outs 
No. 8: Major S3 
No. 9: Major S1 
No. 10: Fullmetal Alchemist: Brotherhood 


**Implementing User Based Collaborative System**


In [ ]:
#Now we use cosine similarity function to get user similarity by inputting the transpose of our pivot sparse table
user_similarity = cosine_similarity(pivot_sparse.T)

In [ ]:
#Inserting the similarity matricies into dataframe objects So that we can perform operations and can further start the main task of recommendation
user_sim_df = pd.DataFrame(user_similarity, index = pivot_norm.columns, columns = pivot_norm.columns)

In [ ]:
#This function will return the top 5 users with the highest similarity value 
# Input : User id  
# Output : list of users with highest similarity
def top_users(user):
    
    if user not in pivot_norm.columns:
        return('No data available on user {}'.format(user))
    
    print('Most Similar Users with user {}:\n'.format(user))
    sim_items = user_sim_df.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:11]
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    zipped = zip(sim_users, sim_items,)
    for user, sim in zipped:
        print('User #{0}, Similarity value: {1:.2f}'.format(user, sim)) 


In [ ]:
# This function calculates the weighted average of similar users to determine a potential rating for an input user and show
def predict_rating(anime_name, user):
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:1000]
    user_values = user_sim_df.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:1000]
    rating_list = []
    weighted_list = []
    for j, i in enumerate(sim_users):
        rating = pivottable.loc[i, anime_name]
        similarity = user_values[j]
        if np.isnan(rating):
            continue
        elif not np.isnan(rating):
            rating_list.append(rating*similarity)
            weighted_list.append(similarity)
    return sum(rating_list)/sum(weighted_list)   

In [ ]:
# Function to make list of highest rated animes per simiar users , returns the anime and its frequency it appears in sorted list
def animes_per_sim_user_freq(user):
    
    if user not in pivot_norm.columns:
        return(' Sorry! There is no data available on user  id {}'.format(user))
    
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    best = []
    most_common = {}
    
    for i in sim_users:
        max_score = pivot_norm.loc[:, i].max()
        best.append(pivot_norm[pivot_norm.loc[:, i]==max_score].index.tolist())
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_list[:5]    

In [ ]:
#Calling out top users function to find similar users based on their similarity values
top_users(3)

Most Similar Users with user 3:

User #2986, Similarity value: 0.32
User #14881, Similarity value: 0.32
User #3028, Similarity value: 0.32
User #18079, Similarity value: 0.32
User #3681, Similarity value: 0.31
User #20855, Similarity value: 0.31
User #9201, Similarity value: 0.31
User #2411, Similarity value: 0.31
User #15062, Similarity value: 0.31
User #12566, Similarity value: 0.30


In [ ]:
#Calling out the function for user id 3
animes_per_sim_user_freq(3)

[('Shingeki no Kyojin', 4),
 ('Sen to Chihiro no Kamikakushi', 4),
 ('Angel Beats!', 3),
 ('Clannad: After Story', 3),
 ('Steins;Gate', 3)]

In [ ]:
# Prediction of rating of  Hajime no Ippo anime by user 3
predict_rating('Hajime no Ippo', 3)

8.862326532708229

**Evaluating the Recommendation System**

To evaluate the system we would be looking at various user-ids and their RMSE's and would also look on coverage metric to draw out conclusions.

In [ ]:
# This function would calculate the coverage for any user present in the dataset of ratings, usually coverage is percentage of the animes that he can be given recommendation.
def  coverage (pred) :
  pred = pred+ 0.001
  pred = pred/predicted_sum
  return -np.sum(pred * np.log2(pred))*100
    

In [ ]:
#Now lets examine how the predict_rating function performs compared to the observed rated values for user 7

watched = pivottable.T[pivottable.loc[7,:]>0].index.tolist()

In [ ]:
#Make a list of the squared errors between actual and predicted value for user id 7
errors = []
predicted_sum=0
for i in watched:
    actual=pivottable.loc[7, i]
    predicted = predict_rating(i,7)
    predicted_sum=predicted_sum+predicted
    errors.append((actual-predicted)**2)

KeyboardInterrupt: ignored

In [ ]:
#This is the average squared error for user id 7
print("RMSE of user-id 7")
np.mean(errors)

In [ ]:
#Calling our coverage function to know what percentage of our anime dataset can be covered by a user through our recommendation system
coverage(predicted)

**Misc examples  of user ids and their Coverage**

In [ ]:
# Calculating errors and finding RMSE for user id: 3
watched = pivottable.T[pivottable.loc[3,:]>0].index.tolist()
errors = []
predicted_sum=0
for i in watched:
    actual=pivottable.loc[3, i]
    predicted = predict_rating(i,3)
    predicted_sum=predicted_sum+predicted
    errors.append((actual-predicted)**2)

print("RMSE of user-id 3")
np.mean(errors)

In [ ]:
#coverage for user ID: 3
coverage(predicted)

In [ ]:
# Calculating errors and finding RMSE for user id: 43
watched = pivottable.T[pivottable.loc[43,:]>0].index.tolist()
errors = []
predicted_sum=0
for i in watched:
    actual=pivottable.loc[43, i]
    predicted = predict_rating(i,43)
    predicted_sum=predicted_sum+predicted
    errors.append((actual-predicted)**2)

print("RMSE of user-id 43")
np.mean(errors)

In [ ]:
#coverage for user ID: 43
coverage(predicted)

In [ ]:
# Calculating errors and finding RMSE for user id: 10
watched = pivottable.T[pivottable.loc[10,:]>0].index.tolist()
errors = []
predicted_sum=0
for i in watched:
    actual=pivottable.loc[10, i]
    predicted = predict_rating(i,10)
    predicted_sum=predicted_sum+predicted
    errors.append((actual-predicted)**2)

print("RMSE of user-id 10")
np.mean(errors)

In [ ]:
#coverage for user ID: 10
coverage(predicted)